# Imports

In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection

# Imports of other project files
from log_wrapper import LogWrapper
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex)

# Other imports
import importlib

import scipy.interpolate

In [ ]:
# Matplotlib configuring
plt.style.use(['science', 'no-latex'])
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Simulation setup
## Pre-finalize steps

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
v_stiction=1e-3
plant.set_stiction_tolerance(v_stiction)
plant.set_penetration_allowance(0.001)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
# def_joint_angles = list(np.zeros(num_links))
# def_joint_angles[0] = np.pi/100
# def_joint_angles[1] = np.pi/100
# def_joint_angles[int(num_links*0.65)-2] = -np.pi/10
# def_joint_angles[int(num_links*0.65)-1] = -np.pi/10
# def_joint_angles[int(num_links*0.65)] = -np.pi/10
# def_joint_angles[int(num_links*0.65)+1] = -np.pi/10
# # def_joint_angles[int(num_links*0.65)+2] = -np.pi/10
def_joint_angles = 0#np.pi/3

paper = Paper(plant, scene_graph, num_links, "NATURAL", default_joint_angle=def_joint_angles,
              stiffness=0,#2.5e-2,
              damping=0,)#7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

finger_instance, finger_body, finger_shape = finger.AddFinger(plant, constants.INIT_Y, constants.INIT_Z)

# Set up logger (needs to happen after all bodies are added)
log_wrapper = LogWrapper(plant.num_bodies(), int(finger_body.index()), paper)
builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
paper.plant.get_body(BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]

## Controller selection

In [ ]:
## CHOOSE CONTROL SYSTEM HERE BY UNCOMMENTING
# # PD control: hits too low
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )

# PD control: hits too high
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )


# Edge feedback
finger_ctrlr = finger.EdgeController(
    plant,
    paper,
    int(finger_instance),
    F_Nd=2,
    debug=True,
)

# # Optimization controller
# finger_ctrlr = finger.OptimizationController(
#     plant,
#     paper,
#     int(finger_instance),
#     paper.get_free_edge_instance()
# )

# Blank controller
# finger_ctrlr = finger.BlankController(
#     plant,
#     int(finger_instance)
# )

## Post-finalize steps

In [ ]:
plant.Finalize()

## Post finalize steps
# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))

# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 
builder.Connect(plant.get_contact_results_output_port(), log_wrapper.get_input_port(3))
builder.Connect(plant.get_reaction_forces_output_port(), log_wrapper.get_input_port(4))

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
if type(finger_ctrlr) is finger.OptimizationController:
    finger_ctrlr.optimize(plant.GetPositions(diagram_context))

# Run simulation

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)

# This way, we can use the actual time the simulation ran for
effective_tspan = max(logger.sample_times())

vis.stop_recording()
vis.publish_recording()

# Plots used in orginal paper

In [ ]:
# This script is getting huge. We need to turn off parts that we're not using.
generate_orig_plots = False

In [ ]:
if generate_orig_plots:
    # Plot manipulator position vs. trajectory
    # PROGRAMMING: Use body index for finger instead of model instance
    if type(finger_ctrlr) is finger.PDFinger:
        plt.figure(figsize=(2*3,2*2))
        plt.plot(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1],
                 logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2],
                 label='Manipulator position')
        plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, label='Trajectory')
        plt.xlabel("$y$ position")
        plt.ylabel("$z$ position")
        plt.legend()
        plt.show()

In [ ]:
if generate_orig_plots:
    # Generate plots for paper
    nb = plant.num_bodies()
    y_traces = []
    z_traces = []
    theta_traces = []
    for b in paper.link_idxs:
        y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
        z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
        theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
    y_traces = np.array(y_traces)
    z_traces = np.array(z_traces)
    theta_traces = np.array(theta_traces)

    times_ = np.arange(0,effective_tspan, effective_tspan/10)
    cmap = cm.get_cmap("viridis_r")
    plt.figure(figsize=(2*3,2*2))
    if type(finger_ctrlr) is finger.PDFinger:
        plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, '--k', zorder=-1)
    for t in times_:
        c = cmap(t/effective_tspan)
        idx = np.argmax(logger.sample_times() >= t)

        # Plot paper
        # PROGRAMMING: Account for paper thickness in plots
        for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
            y0 = y - np.cos(theta)*paper.link_width/2
            z0 = z - np.sin(theta)*paper.link_width/2
            y1 = y + np.cos(theta)*paper.link_width/2
            z1 = z + np.sin(theta)*paper.link_width/2
            plt.plot([y0, y1], [z0, z1], color=c)


        # Plot manipulator
        plt.scatter(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1,idx], 
                    logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2,idx],
                    color=c, s=300, zorder=1)

    xlim = plt.xlim()
    ylim = plt.ylim()
    plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
    plt.xlim(xlim)
    plt.ylim(ylim)
    cb = plt.colorbar()
    cb.set_label("Time")
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.show()

# Drag plots

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
generate_drag_plots = False

In [ ]:
if generate_drag_plots:
    # Plot drag forces
    plt.figure(figsize=(16, 12))
    nb = plant.num_bodies
    vel_traces = []
    for b in paper.link_idxs:
        rho = 1.225
        C_d = paper.link_width*paper.depth
        vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
        f = 0.5*rho*C_d*vel_sqred
        plt.plot(f)

    plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
    plt.show()

# Verifying equations hold
We have the following free body diagrams:
<img src="forces.png" width=400px>
<img src="link-moments.png" width=400px>

Which give us the following equations:
$$
\begin{aligned}
     m_La_{LT} &= F_{FL} + F_{GT} +F_{OT} \\
    m_La_{LN} &= F_{NL} + F_{GN} +F_{ON} \\
    m_Ma_{MT} &= F_{FM} + F_{CT} \\
    m_Ma_{MN} &= F_{NM} + F_{CN} \\
    a_{LT} &= a_{MT} \\
    a_{LN} &= a_{MN} \\
    a_{LT} &= -\frac{w_L}{2}\dot\theta^2 \\
    a_{LN} &= \frac{w_L}{2}\ddot\theta \\
    I_L\ddot\theta &= \frac{w_L}{2}F_{ON}- \frac{h_L}{2}F_{FL} - r_TF_{NL} \\
    F_{NL} &= -F_{NM} \\
    F_{FL} &= -F_{FM} \\
    a_{NL} &= a_{Nd}  \\
    F_{FL} &= 0
\end{aligned}
$$
For each of these equations, I want to plot the value on the left side and the value on the right side to see if they match.

## Loading signals
Preparing all the signals we'll need.

In [ ]:
debug = finger_ctrlr.debug

In [ ]:
l_times = logger.sample_times()
d_times = np.array(debug['times'])

### Inputs
These should all come from simulation/geometry, since they are inputs to the controller and are "fixed" from the controller's perspective.
#### Geometric quantities ($m_L, m_M, w_L, I_L, h_L, \mu$)

In [ ]:
m_L = paper.link_mass
m_M = constants.FINGER_MASS
w_L = paper.link_width
I_L = paper.plant.get_body(
    BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
h_L = paper.height
mu = constants.FRICTION

#### Vectors which aren't determined by the force balance ($F_{GT}, F_{GN} \hat T, \hat N$)
- [ ] PROGRAMMING: also get $\hat T$ from simulation somehow

In [ ]:
N_hats_raw = logger.data()[log_wrapper.contact_entry_start_idx+14:log_wrapper.contact_entry_start_idx+17,:]
N_hats = np.expand_dims(N_hats_raw.T, 2)

In [ ]:
T_hats_no_interp = np.array(debug['T_hats'])
T_hats = scipy.interpolate.interp1d(d_times, T_hats_no_interp, axis=0)(l_times)
T_hats.shape

It's also useful here to calculate project matrices:

In [ ]:
T_hat_proj_mat = np.matmul(T_hats, np.transpose(T_hats, [0, 2, 1]))

In [ ]:
N_hat_proj_mat = np.matmul(N_hats, np.transpose(N_hats, [0, 2, 1]))

- [ ] PROGRAMMING: Calc these directly instead
- [ ] PROGRAMMING: Stop interpolating everything (avoid using `debug`)

In [ ]:
F_GT_no_interp = np.array(debug['F_GTs'])
F_GT = scipy.interpolate.interp1d(d_times, F_GT_no_interp, axis=0)(l_times)
F_GN_no_interp = np.array(debug['F_GNs'])
F_GN = scipy.interpolate.interp1d(d_times, F_GN_no_interp, axis=0)(l_times)

#### Positions and velocities which are "fixed'' until the next timestep ($\dot\theta^2, r_T$)

In [ ]:
d_theta = logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx() + 6 + 3]

Note that $r_T$ here is calculated differently (but more accurately) than in the controller.

In [ ]:
p_contact = np.expand_dims(
    logger.data()[log_wrapper.contact_entry_start_idx+11:log_wrapper.contact_entry_start_idx+14,:].T,
    2)

In [ ]:
p_link = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx():
                  log_wrapper.entries_per_body*paper.get_free_edge_idx()+3].T,
    2)

In [ ]:
p_link_contact = p_contact - p_link
p_link_contact_T_vec = np.matmul(T_hat_proj_mat, p_link_contact)
p_link_contact_T_mag = np.linalg.norm(p_link_contact_T_vec, axis=1)
p_link_contact_T_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    p_link_contact_T_vec))
r_T = p_link_contact_T_mag.flatten()*p_link_contact_T_sgn.flatten()

#### Control inputs ($a_{Nd}$)
- [ ] PROGRAMMING: Pass in `a_Nd` instead of `F_Nd`

In [ ]:
a_Nd = finger_ctrlr.F_Nd/paper.link_mass

### Outputs
These should all come directly from simulation, except for control forces.
#### Accelerations of both objects ($a_{LT}, a_{LN}, a_{MT}, a_{MN}, \ddot\theta$)

In [ ]:
a_link_raw = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12:
                     log_wrapper.entries_per_body*paper.link_idxs[-1]+12+3,:]
a_link = np.expand_dims(a_link_raw.T, 2)

In [ ]:
a_LT_vec = np.matmul(T_hat_proj_mat, a_link)
a_LT_mag = np.linalg.norm(a_LT_vec, axis=1)
a_LT_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    a_LT_vec))
a_LT = a_LT_mag.flatten()*a_LT_sgn.flatten()

In [ ]:
a_LN_vec = np.matmul(N_hat_proj_mat, a_link)
a_LN_mag = np.linalg.norm(a_LN_vec, axis=1)
a_LN_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    a_LN_vec))
a_LN = a_LN_mag.flatten()*a_LN_sgn.flatten()

In [ ]:
a_man_raw = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 12:
                     log_wrapper.entries_per_body*int(finger_body.index())+12+3,:]
a_man = np.expand_dims(a_man_raw.T, 2)

In [ ]:
l_times.shape

In [ ]:
a_MT_vec = np.matmul(T_hat_proj_mat, a_man)
a_MT_mag = np.linalg.norm(a_MT_vec, axis=1)
a_MT_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    a_MT_vec))
a_MT = a_MT_mag.flatten()*a_MT_sgn.flatten()

In [ ]:
a_MN_vec = np.matmul(N_hat_proj_mat, a_man)
a_MN_mag = np.linalg.norm(a_MN_vec, axis=1)
a_MN_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    a_MN_vec))
a_MN = a_MN_mag.flatten()*a_MN_sgn.flatten()

In [ ]:
dd_theta = logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx() + 12 + 3,:]

#### Contact, object, and control forces on each object ($F_{NL}, F_{FL}, F_{ON}, F_{OT}, F_{NM}, F_{FM}, F_{CT}, F_{CN}$

##### Contact

In [ ]:
F_contact_M_raw = logger.data()[log_wrapper.contact_entry_start_idx+0:log_wrapper.contact_entry_start_idx+3,:]
F_contact_M = np.expand_dims(F_contact_M_raw.T, 2)

In [ ]:
F_NM_vec = np.matmul(N_hat_proj_mat, F_contact_M)
F_FM_vec = F_contact_M - F_NM_vec

In [ ]:
F_NM_mag = np.linalg.norm(F_NM_vec, axis=1)
F_NM_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    F_NM_vec))
F_NM = F_NM_mag.flatten()*F_NM_sgn.flatten()

In [ ]:
F_FM_mag = np.linalg.norm(F_FM_vec, axis=1)
F_FM_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    F_FM_vec))
F_FM = F_FM_mag.flatten()*F_FM_sgn.flatten()

In [ ]:
F_FL = -F_FM
F_NL = -F_NM

##### Object

In [ ]:
joint_force_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx:log_wrapper.joint_entry_start_idx+3,:]

In [ ]:
F_OT = joint_force_in_compliance_frame[1,:]
F_ON = joint_force_in_compliance_frame[2,:]

##### Control

In [ ]:
F_CT_no_interp = np.array(debug['F_CTs'])
F_CT = scipy.interpolate.interp1d(d_times, F_CT_no_interp, axis=0)(l_times)
F_CN_no_interp = np.array(debug['F_CNs'])
F_CN = scipy.interpolate.interp1d(d_times, F_CN_no_interp, axis=0)(l_times)

## Plotting settings

In [ ]:
lw = 5

In [ ]:
figsize = (16, 8)

In [ ]:
plot_supporting_plots = False

In [ ]:
t_start = 0.04
t_end = 0.135
idx_start = np.argmax(l_times > t_start)
idx_end = np.argmax(l_times > t_end)

In [ ]:
auto_size = True

In [ ]:
def plot_eq(lhs, lhs_label, rhs, rhs_label, lhs_alpha=1, rhs_alpha=1, lw=5):
    plt.figure(figsize=figsize)
    if isinstance(lhs, float) or isinstance(lhs, int):
        plt.axhline(rhs,
                    label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0, alpha=lhs_alpha)
    else:
        plt.plot(l_times[idx_start:idx_end],
                 lhs[idx_start:idx_end],
                 label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0, alpha=lhs_alpha)
    if isinstance(rhs, float) or isinstance(rhs, int):
        plt.axhline(rhs,
                    label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=rhs_alpha)
    else:
        plt.plot(l_times[idx_start:idx_end],
                 rhs[idx_start:idx_end],
                 label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=rhs_alpha)
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.legend()
    plt.xlabel("Time")
    plt.show()

## Plots

In [ ]:
N_hats.shape

### $m_La_{LT} = F_{FL} + F_{GT} +F_{OT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LT}$'
rhs_label = r'$F_{FL} + F_{GT} +F_{OT}$'

lhs = m_L*a_LT

rhs = F_FL+F_GT+F_OT

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $m_La_{LN} = F_{NL} + F_{GN} +F_{ON}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat N$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LN}$'
rhs_label = r'$F_{NL} + F_{GN} +F_{ON}$'

lhs = m_L*a_LN

rhs = F_NL + F_GN + F_ON

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $m_Ma_{MT} = F_{FM} + F_{CT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the manipulator.

In [ ]:
lhs_label = r'$m_Ma_{MT}$'
rhs_label = r'$F_{FM} + F_{CT}'

lhs = m_M*a_MT

rhs = F_FM + F_CT

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $m_Ma_{MN} = F_{NM} + F_{CN}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat TN direction on the manipulator.

In [ ]:
lhs_label = r'$m_Ma_{MN}$'
rhs_label = r'$ F_{NM} + F_{CN}$'

lhs = m_M*a_MN

rhs = F_NM+F_CN

plot_eq(lhs, lhs_label, rhs, rhs_label)

#### Supporting plot
$F_{CN}= -\frac{F_{GN} w_{L}^{2} - 4 I_{L} a_{Nd} - a_{Nd} m_{L} w_{L}^{2} - 2 a_{Nd} m_{M} r_{T} w_{L} - a_{Nd} m_{M} w_{L}^{2}}{2 r_{T} w_{L} + w_{L}^{2}}$

In [ ]:
lhs_label = r"$F_{CN}$"
rhs_label = r"$-\frac{F_{GN} w_{L}^{2} - 4 I_{L} a_{Nd} - a_{Nd} m_{L} w_{L}^{2} - 2 a_{Nd} m_{M} r_{T} w_{L} - a_{Nd} m_{M} w_{L}^{2}}{2 r_{T} w_{L} + w_{L}^{2}}$"

lhs = F_CN

rhs = -(F_GN*w_L**2 - 4*I_L*a_Nd - a_Nd*m_L*w_L**2 - 2*a_Nd*m_M*r_T*w_L - a_Nd*m_M*w_L**2)/(2*r_T*w_L + w_L**2)

if plot_supporting_plots:
    plot_eq(lhs, lhs_label, rhs, rhs_label)

#### Supporting plot 2

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    plt.plot(l_times, m_M*a_MN, label="$ma$", linewidth=lw, color='lightskyblue', zorder=0)
    plt.plot(l_times, F_NM+F_CN, label="Sum of forces", linewidth=lw, color='b', linestyle='--', zorder=1)
    plt.plot(l_times, F_NM, label="$F_{NM}$", linewidth=lw, linestyle='--', color='red')
    plt.plot(l_times, F_CN, label="$F_{CN}$", linewidth=lw, linestyle='--', color='green')
    plt.ylim(-3, 3)
    plt.legend()
    plt.xlabel("Time")
    plt.show()

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    plt.plot(l_times, m_M*a_MN, label="$ma_{MN}$", linewidth=lw, color='lightskyblue', zorder=0)
    plt.plot(l_times, F_NM+F_CN, label="Sum of forces", linewidth=lw, color='b', linestyle='--', zorder=1)
    plt.plot(l_times, F_NM, label="$F_{NM}$", linewidth=lw, linestyle='--', color='red')
    plt.plot(l_times, F_CN, label="$F_{CN}$", linewidth=lw, linestyle='--', color='green')
    plt.ylim(-3, 3)
    plt.legend()
    plt.xlabel("Time")
    plt.show()

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    plt.plot(l_times, m_M*a_MN-m_L*a_LN, label="$m_Ma_{MN}-m_La_L$", linewidth=lw, color='lightskyblue', zorder=0)
    plt.plot(l_times, F_NM+F_CN-m_L*a_LN, label="Sum of forces - $m_La_L$", linewidth=lw, color='b', linestyle='--', zorder=1)
    plt.plot(l_times, F_NM-m_L*a_LN, label="$F_{NM}-m_La_L$", linewidth=lw, linestyle='--', color='red')
    plt.plot(l_times, F_CN-m_L*a_LN, label="$F_{CN}-m_La_L$", linewidth=lw, linestyle='--', color='green')
    plt.ylim(-3, 3)
    plt.legend()
    plt.xlabel("Time")
    plt.show()

#### Supporting plot 3

In [ ]:
F_NM_from_accel = m_M*a_MN - F_CN

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    plt.plot(l_times, F_NM, label="$F_{NM}$", linewidth=lw, color='lightskyblue', zorder=0)
    plt.plot(l_times, F_NM_from_accel, label="$F_{NM}$ from accel", linewidth=lw, color='b', linestyle='--', zorder=1)
    plt.ylim(-3, -1.5)
    plt.legend()
    plt.xlabel("Time")
    plt.show()

### $a_{LT} = a_{MT}$
*If this plot is correct:*
My condition for static friction is correct.

In [ ]:
lhs_label = r'$a_{LT}$'
rhs_label = r'$a_{MT}$'

lhs = a_LT

rhs = a_MT

plot_eq(lhs, lhs_label, rhs, rhs_label, rhs_alpha=0.5)

It seems reasonable that this plot would be slightly off, because I have been using a strict Coulomb friction model, but there could be some differences because of the approximations used in Drake.

### $a_{LN} = a_{MN}$
*If this plot is correct:*
My condition that the accelerations in the normal direction (since that's the direction they're touching) need to be the same is correct.

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$a_{MN}$'

lhs = a_LN

rhs = a_MN

plot_eq(lhs, lhs_label, rhs, rhs_label)

#### Why don't they match?

Well,  the bodies are not rigid. 

#### Supporting plot

In [ ]:
# man_point = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
#                           log_wrapper.entries_per_body*int(finger_body.index())+3,:]
# man_rot = 

In [ ]:
X_WMs = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
                      log_wrapper.entries_per_body*int(finger_body.index())+6,:].T
X_WLs = logger.data()[log_wrapper.entries_per_body*int(paper.link_idxs[-1]):
                      log_wrapper.entries_per_body*int(paper.link_idxs[-1])+6,:].T

In [ ]:
x_LMs = []
y_LMs = []
z_LMs = []
for X_WM_arr, X_WL_arr in zip(X_WMs, X_WLs):
    X_WM = RigidTransform()
    X_WM.set_rotation(RollPitchYaw(X_WM_arr[3:]))
    X_WM.set_translation(X_WM_arr[:3])
    
    X_WL = RigidTransform()
    X_WL.set_rotation(RollPitchYaw(X_WL_arr[3:]))
    X_WL.set_translation(X_WL_arr[:3])
    X_LM = X_WL.inverse().multiply(X_WM)
    x_LMs.append(X_LM.translation()[0])
    y_LMs.append(X_LM.translation()[1])
    z_LMs.append(X_LM.translation()[2])

In [ ]:
x_LMs = logger.data()[-3]
y_LMs = logger.data()[-2]
z_LMs = logger.data()[-1]

In [ ]:
link_point = logger.data()[log_wrapper.entries_per_body*int(paper.link_idxs[-1]):
                           log_wrapper.entries_per_body*int(paper.link_idxs[-1])+3,:]

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    lw = 5
    plt.gca().add_patch(patches.Rectangle((-w_L/2,-h_L/2),w_L,h_L,linewidth=1,edgecolor='lightgray',facecolor='lightgray'))
    # plt.plot(man_point[1, idx_start:idx_end] - link_point[1, idx_start:idx_end],
    #          man_point[2, idx_start:idx_end] - link_point[2, idx_start:idx_end])
    plt.plot(y_LMs[idx_start:idx_end], z_LMs[idx_start:idx_end])
    plt.show()

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    d_z_LMs = np.diff(z_LMs)/np.diff(l_times)
    dd_z_LMs = np.diff(d_z_LMs)/np.diff(l_times[:-1])
    plt.plot(l_times[:-2][idx_start:idx_end], dd_z_LMs[idx_start:idx_end])
    plt.plot(l_times[idx_start:idx_end], (a_MN-a_LN)[idx_start:idx_end])
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()

In [ ]:
np.array(debug['N_hats'][2000])@np.array(debug['T_hats'][2000]).T + (np.array(debug['N_hats'][2000])@np.array(debug['T_hats'][2000]).T).T

In [ ]:
np.cos(-2*np.arccos(debug['T_hats'][2000][1]))

In [ ]:
np.sin(-2*np.arcsin(debug['T_hats'][2000][2]))

In [ ]:
l_times[2000]

In [ ]:
debug['T_hats'][500]

#### Supporting plot

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    lw = 5
    plt.plot(l_times, a_link[:, 0,:], label='$a_{Lx}$', color='pink', linewidth=lw)
    plt.plot(l_times, a_link[:, 1,:], label='$a_{Ly}$', color='lightgreen', linewidth=lw)
    plt.plot(l_times, a_link[:, 2,:], label='$a_{Lz}$', color='lightskyblue', linewidth=lw)
    plt.plot(l_times, a_man[:,0,:], '--', label='$a_{Mx}$', color='red', linewidth=lw, alpha=0.5)
    plt.plot(l_times, a_man[:,1,:], '--', label='$a_{My}$', color='green', linewidth=lw, alpha=0.5)
    plt.plot(l_times, a_man[:,2,:], '--', label='$a_{Mz}$', color='blue', linewidth=lw, alpha=0.5)
    plt.xlabel("Time (seconds)")
    plt.ylim(-100, 100)
    plt.legend()
    plt.show()

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=figsize)
    lw = 5
    plt.plot(logger.sample_times(), a_link_raw[0, :], label='$a_{Lx}$', color='pink', linewidth=lw)
    plt.plot(logger.sample_times(), a_link_raw[1, :], label='$a_{Ly}$', color='lightgreen', linewidth=lw)
    plt.plot(logger.sample_times(), a_link_raw[2, :], label='$a_{Lz}$', color='lightskyblue', linewidth=lw)
    plt.plot(logger.sample_times(), a_man_raw[0, :], '--', label='$a_{Mx}$', color='red', linewidth=lw, alpha=0.5)
    plt.plot(logger.sample_times(), a_man_raw[1, :], '--', label='$a_{My}$', color='green', linewidth=lw, alpha=0.5)
    plt.plot(logger.sample_times(), a_man_raw[2, :], '--', label='$a_{Mz}$', color='blue', linewidth=lw, alpha=0.5)
    plt.xlabel("Time (seconds)")
    plt.ylim(-100, 100)
    plt.legend()
    plt.show()

### $a_{LT} = -\frac{w_L}{2}\dot\theta^2$

In [ ]:
lhs_label = r'$a_{LT}$'
rhs_label = r'$-\frac{w_L}{2}\dot\theta^2$'

lhs = a_LT

rhs = -(w_L/2)*d_theta**2

plot_eq(lhs, lhs_label, rhs, rhs_label)

#### Supporting plots

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=(16,8))
    plt.plot(l_times[idx_start:idx_end],
                 (-a_LT/d_theta**2)[idx_start:idx_end],
                 label=r"$\frac{a_{LT}}{-\dot\theta^2}$", linewidth=lw, color='skyblue')
    plt.axhline(w_L/2, color='b', linestyle='--', linewidth=5)
    plt.autoscale(enable=True, axis='x', tight=True)
    # plt.ylim(14.4196106, 14.419611)
    plt.legend()
    plt.xlabel("Time")
    plt.show()

### $a_{LN} = \frac{w_L}{2}\ddot\theta$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$\frac{w_L}{2}\ddot\theta$'

lhs = a_LN

rhs = dd_theta*w_L/2

plot_eq(lhs, lhs_label, rhs, rhs_label)

#### Supporting plots

In [ ]:
if plot_supporting_plots:
    plt.figure(figsize=(16,8))
    plt.plot(l_times[idx_start:idx_end],
                 (a_LN/dd_theta)[idx_start:idx_end],
                 label=r"$\frac{\ddot\theta}{a_{LN}}$", linewidth=lw, color='lightskyblue', zorder=1)
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.xlabel("Time")
    # plt.ylim(14.4196106, 14.4196107)
    plt.axhline(w_L/2, color='b', linestyle='--', linewidth=lw, label=r"$\frac{w_L}{2}$")
    plt.axhline(w_L/2 + constants.EPSILON/2, color='navy', linewidth=lw, label=r"$\frac{w_L+\epsilon}{2}$")
    plt.legend()
    plt.show()

### $I_L\ddot\theta = -\frac{w_L}{2}F_{ON}+ \frac{h_L}{2}F_{FL} + r_TF_{NL}$

In [ ]:
lhs_label = r'$I_L\ddot\theta$'
rhs_label = r'$-\frac{w_L}{2}F_{ON}+ \frac{h_L}{2}F_{FL} + r_TF_{NL}$'

lhs = I_L*dd_theta

rhs = -F_ON*w_L/2+h_L*F_FL/2+r_T*F_NL

plot_eq(lhs, lhs_label, rhs, rhs_label)

#### Supporting/debugging plots

In [ ]:
if plot_supporting_plots:
    lw = 5
    plt.figure(figsize=(16, 16))
    plt.plot(l_times, r_T, label="Simulation", linewidth=5, color='lightskyblue', zorder=0)
    plt.plot(d_times, paper.link_width + np.array(debug['d_Ts']) - paper.link_width/2, 
             label="Controller", linewidth=5, color='b', linestyle='--', zorder=1)
    plt.show()

### $F_{NL} = -F_{NM}$

In [ ]:
lhs_label = r'$F_{NL}$'
rhs_label = r'$-F_{NM}$'

lhs = F_NL

rhs = -F_NM

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $F_{FL} = -F_{FM}$

In [ ]:
lhs_label = r'$F_{FL}$'
rhs_label = r'$-F_{FM}$'

lhs = F_FL

rhs = -F_FM

plot_eq(lhs, lhs_label, rhs, rhs_label, rhs_alpha=0.5)

### $a_{LN} = a_{Nd}$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$a_{Nd}$'

lhs = a_LN

rhs = a_Nd

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $F_{FL} = 0$

In [ ]:
lhs_label = r'$F_{FL}$'
rhs_label = r'0'

lhs = F_FL

rhs = 0

plot_eq(lhs, lhs_label, rhs, rhs_label)

# Other useful plots

## Verifying we have the same $\hat N$

In [ ]:
N_hats_sim = N_hats

In [ ]:
N_hats_ctrl = np.array(debug['N_hats'])

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.plot(l_times, N_hats_sim[:,0,:], label='$\hat n_{BA_W,x}$', color='pink', linewidth=lw)
plt.plot(l_times, N_hats_sim[:,1,:], label='$\hat n_{BA_W,y}$', color='lightgreen', linewidth=lw)
plt.plot(l_times, N_hats_sim[:,2,:], label='$\hat n_{BA_W,z}$', color='lightskyblue', linewidth=lw)
plt.plot(d_times, N_hats_ctrl[:,0,:], '--', label='$\hat N_x$', color='red', linewidth=lw)
plt.plot(d_times, N_hats_ctrl[:,1,:], '--', label='$\hat N_y$', color='green', linewidth=lw)
plt.plot(d_times, N_hats_ctrl[:,2,:], '--', label='$\hat N_z$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

The signals line up, so our calculation of $\hat N$ matches what Drake gets.

##  $d_N$ and separation speed

In [ ]:
separation_speed = logger.data()[log_wrapper.contact_entry_start_idx+9,:]

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(d_times, np.array(debug['d_Ns']), label='$d_N$', linewidth=lw)
plt.axhline(-(constants.FINGER_RADIUS), label='Contact $d_N$ (finger radius)', linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

$d_N$ settles at the contact value, which is what we expect.

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times, separation_speed, label="Separation speed", linewidth=lw)
plt.axhline(0, color='k', linestyle='--', label='Contact', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Velocity (m/s)")
plt.show()

Separation speed settles at zero, which is what we expect; means we don't break contact.

## Moving contact point
 - [ ] PROGRAMMING: Add arrow heads to contact point plots

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(d_times, debug['d_Ts'], label='$d_T$', linewidth=lw)
plt.axhline(-(0), label='Bounds on $d_T$ (0 and link width)', linestyle='--', color='k', linewidth=lw)
plt.axhline(-(paper.link_width), linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

This plot shows that $d_T$ changes during the simulation an amount which is significant relative to its bounds.

In [ ]:
contact_point = logger.data()[log_wrapper.contact_entry_start_idx+11:log_wrapper.contact_entry_start_idx+14,:]

Sanity check on contact point data:

In [ ]:
plt.figure(figsize=(12,12))

plt.plot(contact_point[1,:], contact_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

In [ ]:
man_point = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
                          log_wrapper.entries_per_body*int(finger_body.index())+3,:]

In [ ]:
plt.figure(figsize=(12,12))
plt.gca().add_patch(plt.Circle((0, 0), constants.FINGER_RADIUS, color='r'))
plt.plot(contact_point[1,:]-man_point[1,:], contact_point[2,:]-man_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

Shows the contacat point moves around on the surface of the manipulator.

## Friction plots

In [ ]:
slip_speed = logger.data()[log_wrapper.contact_entry_start_idx+10,:]

In [ ]:
t_slip = l_times[np.argmax(slip_speed > v_stiction)]

In [ ]:
lw=5
plt.figure(figsize=(16,12))
plt.plot(l_times, abs(F_FL), label=r"$\left|F_{FL}\right|$", linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(l_times, abs(F_FM), label=r"$\left|F_{ML}\right|$", linewidth=lw, color='b', linestyle='--', zorder=1)
plt.plot(l_times, abs(F_NL)*constants.FRICTION, label=r"$\mu\left|F_{NL}\right|$", linewidth=lw, color='k', linestyle=':')
plt.axvline(t_slip, color='r', label="$t_{slip}$", linewidth=lw)
plt.ylim(-0, 2)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.xlabel("Time")
plt.show()

This plot shows that although we remain within the static friction limit for the majority of the time, at $t_{slip}$ we exceed $v_{stiction}$ and move to dynamic friction.

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.axvline(t_slip, color='r', label="$t_{slip}$", linewidth=lw)
plt.plot(logger.sample_times(), slip_speed, label="slip speed", linewidth=5)
plt.axhline(v_stiction, linestyle="--", color="k", label="$v_{stiction}$", linewidth=5)
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
plt.ylim(0, 2*v_stiction)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

This plot shows what's happening from the slip speed side. Similar to a tire spinning out, as soon as the slip speed exceeds $v_{stiction}$, the positive feedback of lowered friction sends the slip speed quite high.

## Penetration debugging

In [ ]:
lw=5
smooth_idxs = np.arange(800, 4000)
plt.figure(figsize=(16, 8))
plt.plot(l_times, a_MN, linewidth=lw)
plt.plot(l_times[smooth_idxs], a_MN[smooth_idxs], linewidth=lw)
plt.axvline(plant.get_contact_penalty_method_time_scale(), linewidth=lw, color='red')
plt.ylim(-10, 40)
# plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [ ]:
finger_shape

In [ ]:
# scene_graph.model_inspector().GetProximityProperties(finger_shape).GetProperty("material", "point_contact_stiffness");

In [ ]:
pen_depth = logger.data()[log_wrapper.contact_entry_start_idx+17]

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times, pen_depth)
plt.ylim(-2,2)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

In [ ]:
# def get_stiffness_and_damping(penetration_allowance) {
penetration_allowance = 0.001
g = 9.81
mass = 0.0

for bi in range(plant.num_bodies()):
    body = plant.get_body(BodyIndex(bi))
    mass = max(mass, body.get_default_mass())

combined_stiffness = mass * g / penetration_allowance
omega = np.sqrt(combined_stiffness / mass)

time_scale = 1.0 / omega

damping_ratio = 1.0
dissipation = damping_ratio * time_scale / penetration_allowance
stiffness = 2 * combined_stiffness

### Pentration into manipulator

In [ ]:
pen_man_point = pen_depth = logger.data()[log_wrapper.contact_entry_start_idx+3:
                                         log_wrapper.contact_entry_start_idx+6]

In [ ]:
np.linalg.norm(pen_man_point[1:3:]-man_point[1:3,:], axis=0).shape

In [ ]:
depth_man = np.linalg.norm(pen_man_point[1:3:]-man_point[1:3,:], axis=0)

In [ ]:
l_times.shape

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(l_times, depth_man, color='k', linestyle='-', linewidth='5')
ylims = plt.ylim()
plt.ylim(0,ylims[-1]*2)
plt.show()

### Penetration into link

In [ ]:
link_point = logger.data()[log_wrapper.entries_per_body*int(paper.link_idxs[-1]):
                           log_wrapper.entries_per_body*int(paper.link_idxs[-1])+3,:]

In [ ]:
pen_link_point = logger.data()[log_wrapper.contact_entry_start_idx+6:
                               log_wrapper.contact_entry_start_idx+9]

In [ ]:
depth_link = np.linalg.norm(pen_link_point[1:3:]-link_point[1:3,:], axis=0)

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(l_times, depth_man, color='k', linestyle='-', linewidth='5')
ylims = plt.ylim()
plt.ylim(0,ylims[-1]*2)
plt.show()

## Accel debugging

In [ ]:
v_man_diff = np.diff(man_point[1:3,:])/np.diff(l_times)
a_man_diff = np.diff(v_man_diff)/np.diff(l_times[:-1])

In [ ]:
lw=5
plt.figure(figsize=(16,16))
for i in range(1,3):
    plt.subplot(2,1,i)
    plt.plot(l_times[:-2][::10], a_man_diff[i-1,:][::10], linewidth=lw)
    plt.plot(l_times, a_man[:,i,0], linewidth=lw)
    plt.ylim(-150, 150)
    plt.legend()
    plt.xlabel("Time")
plt.show()

In [ ]:
v_contact_diff = np.diff(contact_point[1:3,:])/np.diff(l_times)
a_contact_diff = np.diff(v_contact_diff)/np.diff(l_times[:-1])

In [ ]:
a_contact_diff.shape

In [ ]:
lw=5
plt.figure(figsize=(16,16))
for i in range(1,3):
    plt.subplot(2,1,i)
    plt.plot(l_times[:-2][::10], a_contact_diff[i-1,:][::10], linewidth=lw)
    plt.plot(l_times, a_man[:,i,0], linewidth=lw)
    plt.ylim(-150, 150)
    plt.legend()
    plt.xlabel("Time")
plt.show()